# Google Analytics 360 Sample

Analysis of ecommerce data from Google Analytics 360 exported to BigQuery from the Google Merchandise Store. 

This exercise is intended to demonstrate an understanding of 360 data as well as the use of Google's BigQuery cloud database. 

Google has made this data availble as a [public dataset](https://cloud.google.com/bigquery/public-data/).

**Where it comes from**

The sample dataset contains obfuscated Google Analytics 360 data from the [Google Merchandise Store](https://www.googlemerchandisestore.com/shop.axd/Home?utm_source=Partners&utm_medium=affiliate&utm_campaign=Data%20Share%20Promo), a real ecommerce store. The Google Merchandise Store sells Google branded merchandise. The data is typical of what you would see for an ecommerce website. It includes the following kinds of information:

- Traffic source data: information about where website visitors originate. This includes data about organic traffic, paid search traffic, display traffic, etc.
- Content data: information about the behavior of users on the site. This includes the URLs of pages that visitors look at, how they interact with content, etc.
- Transactional data: information about the transactions that occur on the Google Merchandise Store website.

Data includes records from August of 2016 through August of 2017. 


--- 

Import dependancies and create Google Cloud client connection to my project through a BigQuery service account. 

In [1]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
import plotly.graph_objects as go
import fohr_theme_light
import chart_studio.plotly as py
pd.options.plotting.backend = 'plotly'

# Connect to 
CREDENTIALS = service_account.Credentials.from_service_account_file('bigquery.json')
PROJECT_ID = 'udacity-bigquery'
CLIENT = bigquery.Client(credentials=CREDENTIALS, project=PROJECT_ID)

print(f'numpy = {np.__version__}')
print(f'pandas = {pd.__version__}')
print(f'bigquery = {bigquery.__version__}')

numpy = 1.19.1
pandas = 1.1.0
bigquery = 1.22.0


In [2]:
CREDENTIALS.service_account_email

'bigquery@udacity-bigquery.iam.gserviceaccount.com'

---

Let's take a look at what is stored in this data frame. Similar to AWS Athena, Google BigQuery has subtables inside columns. It can also auto-detect a schema on read for supported formats and allows for Nulls. 

In [3]:
query = '''
    SELECT distinct column_name, data_type
    FROM `bigquery-public-data.google_analytics_sample.INFORMATION_SCHEMA.COLUMNS`
    WHERE table_name like 'ga_sessions_%'
    ORDER BY 1
'''
data = CLIENT.query(query).result().to_dataframe()
data

,column_name,data_type
0,channelGrouping,STRING
1,clientId,STRING
2,customDimensions,"ARRAY<STRUCT<index INT64, value STRING>>"
3,date,STRING
4,device,"STRUCT<browser STRING, browserVersion STRING, ..."
5,fullVisitorId,STRING
6,geoNetwork,"STRUCT<continent STRING, subContinent STRING, ..."
7,hits,"ARRAY<STRUCT<hitNumber INT64, time INT64, hour..."
8,hits,"ARRAY<STRUCT<hitNumber INT64, time INT64, hour..."
9,socialEngagementType,STRING


I can see that 7 of the 17 columns have a sub-structure with more data. Using a SQL-like sintax of `.` we can ultimate call these sub-column fields directly.

Let's take a look at the first one.

In [4]:
query = f''' 
    SELECT fullVisitorId, d.* 
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`, 
    UNNEST(customDimensions) d 
    WHERE _TABLE_SUFFIX between '20160801' and '20160901' 
    LIMIT 5 
    '''
data = CLIENT.query(query).result().to_dataframe()
data

,fullVisitorId,index,value
0,7598107164796351257,4,APAC
1,5909713377620782980,4,Central America
2,1119565409681207118,4,North America
3,2151631661398435396,4,EMEA
4,1675396657302736678,4,North America


Hmmm.  I don't really know what `index` means. `value` is clearly a geo region, but it looks like that might also be stored in the `GeoNetwork` column.

Let's look at another column.

In [5]:
query = f''' 
    SELECT visitorId, fullVisitorId, device.* 
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
    WHERE _TABLE_SUFFIX between '20160801' and '20160901' 
    LIMIT 5 
    '''
data = CLIENT.query(query).result().to_dataframe()
data

,visitorId,fullVisitorId,browser,browserVersion,browserSize,operatingSystem,operatingSystemVersion,isMobile,mobileDeviceBranding,mobileDeviceModel,mobileInputSelector,mobileDeviceInfo,mobileDeviceMarketingName,flashVersion,javaEnabled,language,screenColors,screenResolution,deviceCategory
0,None,9621622720764337033,Safari,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,None,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop
1,None,7600379367948870835,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,None,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop
2,None,0594549172933846354,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,None,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop
3,None,9595629889938024111,Firefox,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,None,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop
4,None,5748628212328826505,Firefox,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,None,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop


In [6]:
data.shape[1]

19

Now that is a lot more information. The sub-fields in the `device` columns allows us to see information about the device connecting to the store. However, as this is a public dataset, much of the data has been removed. Many columns have `not available in demo dataset` as the only value in all fields. `browser`, `operatingSystem`, `isMobile` and `deviceCategory` remain. Although the last two seem to be redundant information.

---



In [7]:
query = f''' 
    SELECT visitorId, fullVisitorId, geoNetwork.* 
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
    WHERE _TABLE_SUFFIX between '20160801' and '20160901' 
    LIMIT 5 
    '''
data = CLIENT.query(query).result().to_dataframe()
data

,visitorId,fullVisitorId,continent,subContinent,country,region,metro,city,cityId,networkDomain,latitude,longitude,networkLocation
0,None,9621622720764337033,Asia,Western Asia,Turkey,Istanbul,(not set),Istanbul,not available in demo dataset,ttnet.com.tr,not available in demo dataset,not available in demo dataset,not available in demo dataset
1,None,7600379367948870835,Americas,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,rr.com,not available in demo dataset,not available in demo dataset,not available in demo dataset
2,None,0594549172933846354,Europe,Eastern Europe,Russia,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,tvoe.tv,not available in demo dataset,not available in demo dataset,not available in demo dataset
3,None,9595629889938024111,Asia,Eastern Asia,China,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset
4,None,5748628212328826505,Americas,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,metrocast.net,not available in demo dataset,not available in demo dataset,not available in demo dataset


`geoNetwork` looks to be infomation about the location of the user. Only `latitude`, `longitude` and `networkLocation` have been removed. 

---


In [8]:
query = f''' 
    SELECT fullVisitorId, h.* 
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`, 
    UNNEST(hits) h
    WHERE _TABLE_SUFFIX between '20160801' and '20160901' 
    LIMIT 50
    '''
data = CLIENT.query(query).result().to_dataframe()
data.head()

,fullVisitorId,hitNumber,time,hour,minute,isSecure,isInteraction,isEntrance,isExit,referer,...,customVariables,customDimensions,customMetrics,type,social,latencyTracking,sourcePropertyInfo,contentGroup,dataSource,publisher_infos
0,920561211218448092,1,0,20,56,None,True,True,True,https://www.google.com.au/,...,[],[],[],PAGE,"{'socialInteractionNetwork': None, 'socialInte...",None,None,"{'contentGroup1': '(not set)', 'contentGroup2'...",None,[]
1,8358392293870842399,1,0,7,43,None,True,True,True,None,...,[],[],[],PAGE,"{'socialInteractionNetwork': None, 'socialInte...",None,None,"{'contentGroup1': '(not set)', 'contentGroup2'...",None,[]
2,073334216298189757,1,0,3,50,None,True,True,True,None,...,[],[],[],PAGE,"{'socialInteractionNetwork': None, 'socialInte...",None,None,"{'contentGroup1': '(not set)', 'contentGroup2'...",None,[]
3,4472401511891696800,1,0,9,5,None,True,True,True,https://www.google.com,...,[],[],[],PAGE,"{'socialInteractionNetwork': None, 'socialInte...",None,None,"{'contentGroup1': '(not set)', 'contentGroup2'...",None,[]
4,1626562199419361908,1,0,2,44,None,True,True,True,https://www.google.ch/,...,[],[],[],PAGE,"{'socialInteractionNetwork': None, 'socialInte...",None,None,"{'contentGroup1': '(not set)', 'contentGroup2'...",None,[]


In [9]:
data.shape[1]

34

In [10]:
data.columns

Index(['fullVisitorId', 'hitNumber', 'time', 'hour', 'minute', 'isSecure',
       'isInteraction', 'isEntrance', 'isExit', 'referer', 'page',
       'transaction', 'item', 'contentInfo', 'appInfo', 'exceptionInfo',
       'eventInfo', 'product', 'promotion', 'promotionActionInfo', 'refund',
       'eCommerceAction', 'experiment', 'publisher', 'customVariables',
       'customDimensions', 'customMetrics', 'type', 'social',
       'latencyTracking', 'sourcePropertyInfo', 'contentGroup', 'dataSource',
       'publisher_infos'],
      dtype='object')

In [11]:
data['page'].values[4]

{'pagePath': '/home',
 'hostname': 'www.googlemerchandisestore.com',
 'pageTitle': 'Google Online Store',
 'searchKeyword': None,
 'searchCategory': None,
 'pagePathLevel1': '/home',
 'pagePathLevel2': '',
 'pagePathLevel3': '',
 'pagePathLevel4': ''}

In [12]:
data['referer'].head()

0    https://www.google.com.au/
1                          None
2                          None
3        https://www.google.com
4        https://www.google.ch/
Name: referer, dtype: object

34 sub-fields of data in the `hits` column. Lots of great information in this column. However, like before many pieces have been removed. 

In [13]:
query = f''' 
    SELECT visitorId, fullVisitorId, totals.* 
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
    WHERE _TABLE_SUFFIX between '20160801' and '20160901' 
    LIMIT 5 
    '''
data = CLIENT.query(query).result().to_dataframe()
data

,visitorId,fullVisitorId,visits,hits,pageviews,timeOnSite,bounces,transactions,transactionRevenue,newVisits,screenviews,uniqueScreenviews,timeOnScreen,totalTransactionRevenue,sessionQualityDim
0,None,7598107164796351257,1,1,1,None,1,None,None,1,None,None,None,None,None
1,None,5909713377620782980,1,1,1,None,1,None,None,1,None,None,None,None,None
2,None,1119565409681207118,1,1,1,None,1,None,None,1,None,None,None,None,None
3,None,2151631661398435396,1,1,1,None,1,None,None,1,None,None,None,None,None
4,None,2004769516783991626,1,1,1,None,1,None,None,1,None,None,None,None,None


In [14]:
data.shape[1]

15

15 subfields in the `totals` column. Key here will be `transactions` and `transactionRevenue`.

In [15]:
query = f''' 
    SELECT trafficSource.* 
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
    WHERE _TABLE_SUFFIX between '20160801' and '20160901' 
    LIMIT 50
    '''
data = CLIENT.query(query).result().to_dataframe()
data.head()

,referralPath,campaign,source,medium,keyword,adContent,adwordsClickInfo,isTrueDirect,campaignCode
0,None,(not set),google,organic,(not provided),None,"{'campaignId': None, 'adGroupId': None, 'creat...",True,None
1,None,(not set),google,organic,(not provided),None,"{'campaignId': None, 'adGroupId': None, 'creat...",True,None
2,None,(not set),(direct),(none),None,None,"{'campaignId': None, 'adGroupId': None, 'creat...",True,None
3,None,(not set),google,organic,(not provided),None,"{'campaignId': None, 'adGroupId': None, 'creat...",None,None
4,None,(not set),google,organic,(not provided),None,"{'campaignId': None, 'adGroupId': None, 'creat...",True,None


In [16]:
data.shape[1]

9

In [17]:
data['adwordsClickInfo'].values[42]

{'campaignId': None,
 'adGroupId': None,
 'creativeId': None,
 'criteriaId': None,
 'page': None,
 'slot': None,
 'criteriaParameters': 'not available in demo dataset',
 'gclId': None,
 'customerId': None,
 'adNetworkType': None,
 'targetingCriteria': None,
 'isVideoAd': None}

The `trafficSource` column also has interesting sub-fields to look at. Sadly it doesn't seem that `adwordsClickInfo` is one of them. 

---

Finally lets look at the direct columns to see the type of data involved and better understand its meaning in context. 

In [18]:
query = f''' 
    SELECT  fullvisitorId, channelGrouping, clientId, socialEngagementType, userId, visitId, visitNumber, visitStartTime, visitorId	
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
    WHERE _TABLE_SUFFIX between '20170701' and '20170801'
    LIMIT 50
    '''
data = CLIENT.query(query).result().to_dataframe()

In [19]:
data.sample(5)

,fullvisitorId,channelGrouping,clientId,socialEngagementType,userId,visitId,visitNumber,visitStartTime,visitorId
33,5351779873412793500,Organic Search,None,Not Socially Engaged,None,1500201625,1,1500201625,None
15,8427834733388537860,Direct,None,Not Socially Engaged,None,1500258307,1,1500258307,None
36,2367696628094623267,Organic Search,None,Not Socially Engaged,None,1500230597,1,1500230597,None
30,1505241938886140895,Organic Search,None,Not Socially Engaged,None,1500269334,1,1500269334,None
41,1239322026227601116,Organic Search,None,Not Socially Engaged,None,1500192670,1,1500192670,None


Interestingly we see that `fullvisitorId` number 0288998128011317800 has `7` listed in visitNumber.  It is assumned that we have seen visitor: 0288998128011317800 at least 7 times.

Let's check the data to see if that means what I think it means. Let's pull the query back the full year and look for that `fullvisitorId` to see if we have more than one record and if the `visitNumber` increments accordingly. 

Understand that this visitor may have come to the site before the August 2016, but perhaps we can still see the visitor count increment. 

In [20]:
query = f''' 
    SELECT  fullvisitorId, channelGrouping, clientId, socialEngagementType, userId, visitId, visitNumber, visitStartTime, visitorId	
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
    WHERE _TABLE_SUFFIX between '20160801' and '20170801' and fullvisitorId = '0288998128011317800'
    LIMIT 50
    '''
data = CLIENT.query(query).result().to_dataframe()

In [21]:
data

,fullvisitorId,channelGrouping,clientId,socialEngagementType,userId,visitId,visitNumber,visitStartTime,visitorId
0,0288998128011317800,Organic Search,None,Not Socially Engaged,None,1498637120,6,1498637120,None
1,0288998128011317800,Organic Search,None,Not Socially Engaged,None,1491825966,1,1491825966,None
2,0288998128011317800,Organic Search,None,Not Socially Engaged,None,1498978602,7,1498979704,None
3,0288998128011317800,Organic Search,None,Not Socially Engaged,None,1498978602,7,1498978602,None
4,0288998128011317800,Organic Search,None,Not Socially Engaged,None,1494753995,5,1494753995,None
5,0288998128011317800,Organic Search,None,Not Socially Engaged,None,1492967657,4,1492967657,None
6,0288998128011317800,Organic Search,None,Not Socially Engaged,None,1500035030,8,1500035030,None
7,0288998128011317800,Organic Search,None,Not Socially Engaged,None,1492085541,2,1492085541,None
8,0288998128011317800,Organic Search,None,Not Socially Engaged,None,1492506599,3,1492506599,None


Indeed we can see that user has come to the size at least 8 times during the year we have data for. This confirms that we can use `visitNumber` in the way I assumed to help analyse the data.

---

Following this test, I will also test to be sure that dates also line up with this and look for transactionTotal amounts to determine if these are cumulative for the lifetime of the visitor.

First I will need to find a visitor that has many visit and at least a couple of trnasactions.



In [22]:
query = f''' 
    SELECT  fullvisitorId, AVG(visitNumber), sum(totals.transactions), sum(totals.transactionRevenue), sum(totals.totalTransactionRevenue)
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
    WHERE _TABLE_SUFFIX between '20160801' and '20170801' and visitNumber > 1 and channelGrouping = 'Display'
    GROUP BY fullvisitorId
    LIMIT 50
    '''
data = CLIENT.query(query).result().to_dataframe()

In [23]:
data.loc[data['f1_'].notna()]

,fullvisitorId,f0_,f1_,f2_,f3_
24,7272617006657421843,5.500000,1.0,40550000.0,49550000.0
29,0220695720492664721,3.666667,1.0,46370000.0,54370000.0
37,2549182723988683065,2.500000,1.0,187650000.0,188650000.0


In [24]:
query = f''' 
    SELECT  fullvisitorId, date, visitId, visitNumber, visitStartTime, visitorId, totals.transactions, totals.transactionRevenue, totals.totalTransactionRevenue
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
    WHERE _TABLE_SUFFIX between '20160801' and '20170801' and fullvisitorId = '4085972867127024343'
    LIMIT 50
    '''
data = CLIENT.query(query).result().to_dataframe()

In [25]:
data.sort_values('visitNumber')

,fullvisitorId,date,visitId,visitNumber,visitStartTime,visitorId,transactions,transactionRevenue,totalTransactionRevenue
0,4085972867127024343,20161014,1476453496,1,1476453496,None,NaN,NaN,NaN
9,4085972867127024343,20161101,1478033380,2,1478033380,None,NaN,NaN,NaN
4,4085972867127024343,20161107,1478547833,3,1478547833,None,NaN,NaN,NaN
5,4085972867127024343,20161107,1478550684,4,1478550684,None,1.0,19190000.0,26190000.0
6,4085972867127024343,20161110,1478808651,7,1478808651,None,NaN,NaN,NaN
2,4085972867127024343,20161113,1479097793,8,1479097793,None,1.0,19190000.0,24190000.0
7,4085972867127024343,20161120,1479676039,9,1479676039,None,NaN,NaN,NaN
1,4085972867127024343,20161121,1479760240,10,1479760240,None,NaN,NaN,NaN
8,4085972867127024343,20161122,1479856417,11,1479856417,None,1.0,23990000.0,29990000.0
3,4085972867127024343,20161213,1481660216,12,1481660216,None,1.0,19190000.0,22190000.0


Great. Now we can see that `totalTransactionRevenue` is not cumalitive across a visitors lifetime. In fact it looks like the value is incrementally higher than the `transactionRevenue` column. I think we can assume that a `totalTransactionRevenue` is something like tax or shipping costs.

----

Now, can Transactions ever be more than 1 in a single record?  Meaning do transactions represent items in an order? 

In [26]:
query = f''' 
    SELECT totals.transactions
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
    WHERE _TABLE_SUFFIX between '20160801' and '20170801' and totals.transactions > 1
    LIMIT 50
    '''
data = CLIENT.query(query).result().to_dataframe()

In [27]:
data.head()

,transactions
0,2
1,2
2,2
3,5
4,2


Indeed they do.  

Now I know that transactions must have represent the number of items in an order. We can use this to look for visitors that purchase more than one item. 

## Three Questions of the data

1. Do macOS/iOS affected by traffic source in the same way... organic, paid display. Are targeted ads less effective on iOS? 

2. Do macOS/iOS users purchase more than windows/android users?

3. Is there a correlation of refering traffic to higher average order value?



--- 

First give a sense of how much traffic is coming to the site. 

In [28]:
query = f''' 
    SELECT device.operatingSystem, count(device.operatingSystem)
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
    GROUP BY device.operatingSystem
    ;
    '''
data = CLIENT.query(query).result().to_dataframe()

In [29]:
data = data.loc[data['f0_']>5000].sort_values('f0_', ascending=False)

In [30]:
data['f0_'] = data['f0_'].div(data['f0_'].sum(axis=0))

In [31]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        y=data['f0_'],
        x=data['operatingSystem']
    )
)
fig.update_layout(title='Traffic by Operating System')
fig.update_yaxes(tickformat=',.0%',)
fig.show(renderer='notebook_connected')

In [32]:
# Send Figure to Plotly Chart Studio to share on embed
# py.plot(fig, filename = 'udacity-web-traffic', sharing='public')

In [33]:
query = f''' 
    SELECT channelGrouping, device.operatingSystem
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
    ;
    '''
data = CLIENT.query(query).result().to_dataframe()

In [34]:
# Turn traffic source into dummy categories
data = data.drop('channelGrouping', axis=1).join(pd.get_dummies(data['channelGrouping']))
# Add total volume of traffic
totals = data.groupby('operatingSystem').sum().sum(axis=1)
# Get percentage of traffice from soruce
data = data.groupby('operatingSystem').sum().div(data.groupby('operatingSystem').sum().sum(axis=1), axis=0)
# Add totals onto DataFrame
data['totals'] = totals
# Sort Descending
data.sort_values('totals', ascending=False, inplace=True)
# Remove minor Operating Systems like Xbox and Windows Phone.
data = data.loc[data['totals']>5000]

In [35]:
data

,(Other),Affiliates,Direct,Display,Organic Search,Paid Search,Referral,Social,totals
operatingSystem,,,,,,,,,
Windows,0.000080,0.029762,0.114999,0.003222,0.416329,0.017956,0.065587,0.352065,350072.0
Macintosh,0.000165,0.015811,0.134419,0.009372,0.315431,0.020619,0.221790,0.282392,253938.0
Android,0.000178,0.008919,0.261163,0.007781,0.506433,0.051214,0.028872,0.135441,123892.0
iOS,0.000139,0.003920,0.190991,0.006799,0.658115,0.058357,0.018734,0.062945,107665.0
Linux,0.000257,0.009391,0.250243,0.018325,0.241480,0.005652,0.376063,0.098590,35034.0
Chrome OS,0.000114,0.003455,0.190910,0.015643,0.458784,0.036602,0.255344,0.039146,26337.0


In [36]:
# Reorder columns based on volume
data = data[list(data.sum(axis=0).sort_values(ascending=False).index)]

In [37]:
fig = go.Figure()

for i in data.columns[1:-2]:
    fig.add_trace(
        go.Bar(
            x=data.index,
            y=data[i],
            name = i,
            hovertext=data['totals']
        )
    )
fig.update_layout(title='Refering Source by Operating System')
fig.update_yaxes(tickformat=',.0%',)
fig.show(renderer='notebook_connected')

In [38]:
# Send Figure to Plotly Chart Studio to share on embed
# py.plot(fig, filename = 'udacity-web-traffic_by_source', sharing='public')

That's a little busy.  Let's boil this down to paid and unpaid and show just what we are looking for. 

In [39]:
data['Unpaid'] = data['Organic Search'] + data['Direct'] + data['Referral'] + data['Social']
data['Paid'] = data['Paid Search'] + data['Affiliates'] + data['Display']

<ipython-input-39-9f6d69e33808>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-39-9f6d69e33808>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [40]:
data

,totals,Organic Search,Direct,Social,Referral,Paid Search,Affiliates,Display,(Other),Unpaid,Paid
operatingSystem,,,,,,,,,,,
Windows,350072.0,0.416329,0.114999,0.352065,0.065587,0.017956,0.029762,0.003222,0.000080,0.948979,0.050941
Macintosh,253938.0,0.315431,0.134419,0.282392,0.221790,0.020619,0.015811,0.009372,0.000165,0.954032,0.045803
Android,123892.0,0.506433,0.261163,0.135441,0.028872,0.051214,0.008919,0.007781,0.000178,0.931908,0.067914
iOS,107665.0,0.658115,0.190991,0.062945,0.018734,0.058357,0.003920,0.006799,0.000139,0.930785,0.069075
Linux,35034.0,0.241480,0.250243,0.098590,0.376063,0.005652,0.009391,0.018325,0.000257,0.966376,0.033368
Chrome OS,26337.0,0.458784,0.190910,0.039146,0.255344,0.036602,0.003455,0.015643,0.000114,0.944185,0.055701


In [41]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=data.index,
        y=data['Paid']
    )
)
fig.update_layout(title='Paid Traffic by Operating System')
fig.update_yaxes(tickformat=',.0%',)
fig.show(renderer='notebook_connected')

In [42]:
# Send Figure to Plotly Chart Studio to share on embed
# py.plot(fig, filename = 'udacity-paid_web-traffic', sharing='public')

## Did visitors who came to the site via paid source have a higher rate of transaction? 

First lets check the data to test that we can see the source as a result of a transaction. The key point is to see if the data maintains the value of source even on second and third visits. Perhaps the user was brought to the site through a piad visit initially and subsequently made a purchase on another visit as a direct visitor. 

In [43]:
query = f''' 
    SELECT  fullvisitorId, channelGrouping,  device.operatingSystem, date, visitId, visitNumber, trafficSource.source, trafficSource.medium, totals.transactions, totals.transactionRevenue
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
    WHERE _TABLE_SUFFIX between '20160801' and '20170801' and fullvisitorId = '4085972867127024343'
    LIMIT 50
    '''
data = CLIENT.query(query).result().to_dataframe()

In [44]:
data

,fullvisitorId,channelGrouping,operatingSystem,date,visitId,visitNumber,source,medium,transactions,transactionRevenue
0,4085972867127024343,Referral,Macintosh,20161121,1479760240,10,(direct),(none),NaN,NaN
1,4085972867127024343,Referral,Macintosh,20161110,1478808651,7,(direct),(none),NaN,NaN
2,4085972867127024343,Referral,Macintosh,20161014,1476453496,1,(direct),(none),NaN,NaN
3,4085972867127024343,Referral,Macintosh,20161113,1479097793,8,(direct),(none),1.0,19190000.0
4,4085972867127024343,Referral,Macintosh,20161107,1478547833,3,(direct),(none),NaN,NaN
5,4085972867127024343,Referral,Macintosh,20161107,1478550684,4,(direct),(none),1.0,19190000.0
6,4085972867127024343,Referral,Macintosh,20161122,1479856417,11,(direct),(none),1.0,23990000.0
7,4085972867127024343,Referral,Macintosh,20161101,1478033380,2,(direct),(none),NaN,NaN
8,4085972867127024343,Referral,Macintosh,20161120,1479676039,9,(direct),(none),NaN,NaN
9,4085972867127024343,Referral,Macintosh,20161213,1481660216,12,(direct),(none),1.0,19190000.0


When looking at our case above, we can see that the channel remains the same throughout all visits. Let's check another account with transactions and multiple visits. 

In [45]:
query = f''' 
    SELECT  fullvisitorId, channelGrouping, device.operatingSystem, date, trafficSource.source, trafficSource.medium, totals.transactions, totals.transactionRevenue
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
    WHERE _TABLE_SUFFIX between '20160801' and '20170801' and fullvisitorId = '4140559214164857738'
    LIMIT 50
    '''
data = CLIENT.query(query).result().to_dataframe()

In [46]:
data

,fullvisitorId,channelGrouping,operatingSystem,date,source,medium,transactions,transactionRevenue
0,4140559214164857738,Organic Search,Macintosh,20170726,google,organic,1,119400000
1,4140559214164857738,Organic Search,Macintosh,20170613,(direct),(none),1,117340000
2,4140559214164857738,Organic Search,Macintosh,20170707,google,organic,1,243660000
3,4140559214164857738,Organic Search,Macintosh,20170606,(direct),(none),1,120250000


Again the channel grouping looks to be maintained through all visits. One more. 

In [47]:
query = f''' 
    SELECT  fullvisitorId, channelGrouping, date, visitId, visitNumber, visitStartTime, trafficSource.source, trafficSource.medium, totals.transactions, totals.transactionRevenue, totals.totalTransactionRevenue
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
    WHERE _TABLE_SUFFIX between '20160801' and '20170801' and fullvisitorId = '6377171440951916612'
    LIMIT 50
    '''
data = CLIENT.query(query).result().to_dataframe()

In [48]:
data

,fullvisitorId,channelGrouping,date,visitId,visitNumber,visitStartTime,source,medium,transactions,transactionRevenue,totalTransactionRevenue
0,6377171440951916612,Paid Search,20170725,1501037798,3,1501037798,google,cpc,None,None,None
1,6377171440951916612,Paid Search,20170725,1501039074,4,1501039074,google,cpc,None,None,None
2,6377171440951916612,Organic Search,20170716,1500238874,2,1500238874,google,organic,None,None,None
3,6377171440951916612,Organic Search,20170511,1494556236,1,1494556236,google,organic,None,None,None


Ok, Now we can see that this user has multiple types of traffic source. 

In [49]:
query = f''' 
    SELECT  fullvisitorId, channelGrouping, date, visitId, visitNumber, visitStartTime, trafficSource.source, trafficSource.medium, totals.transactions, totals.transactionRevenue, totals.totalTransactionRevenue
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
    WHERE _TABLE_SUFFIX between '20160801' and '20170801' and fullvisitorId = '738777585657711314'
    LIMIT 50
    '''
data = CLIENT.query(query).result().to_dataframe()

In [50]:
data

,fullvisitorId,channelGrouping,date,visitId,visitNumber,visitStartTime,source,medium,transactions,transactionRevenue,totalTransactionRevenue
0,738777585657711314,Display,20161220,1482262245,3,1482262245,dfa,cpm,NaN,NaN,NaN
1,738777585657711314,Display,20161220,1482267318,4,1482267318,dfa,cpm,NaN,NaN,NaN
2,738777585657711314,Display,20161220,1482272043,5,1482272043,dfa,cpm,1.0,79130000.0,86130000.0
3,738777585657711314,Display,20161228,1482955195,6,1482955195,dfa,cpm,NaN,NaN,NaN
4,738777585657711314,Display,20161219,1482183763,2,1482183763,dfa,cpm,NaN,NaN,NaN


## Before we get too far along, perhaps it would be good to see who is performing tansactions. 

Lets diferentiate between those visitors who have come to the site in any fashion through a targeted ad. 

In [51]:
query = f''' 
    SELECT fullvisitorId,
           operatingSystem,
           sum(paid),
           max(visitNumber),
           sum(transactions),
           sum(transactionRevenue)/1000000
    FROM (SELECT  fullvisitorId, 
            device.operatingSystem,
            CASE WHEN channelGrouping in ('Affliates', 'Display', 'Paid Search') THEN 1 ELSE 0 end as paid, 
            visitNumber,  
            totals.transactions, 
            totals.transactionRevenue,
          FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
          -- To maintain consistency, remove inconsiquential Operating systems.
          WHERE device.operatingSystem in ('Windows', 'Macintosh', 'iOS', 'Android', 'Linux', 'Chrome OS'))
    GROUP BY 1, 2
    '''
data = CLIENT.query(query).result().to_dataframe()

Aggregate this data to see the sucess rate of purchases between users who experienced at least one paid ad. 

First lets inspect the data. 

In [52]:
# Rename columns to something readable
data.columns = ['fullvisitorId', 'operatingSystem', 'Paid', 'Visits', 'Transactions', 'Revenue']
# Mark Paid column to a Boolean
data['Paid'] = data['Paid']>1

In [53]:
data.describe()

,Visits,Transactions,Revenue
count,708258.000000,10021.000000,9995.000000
mean,1.324242,1.208861,154.081525
std,1.977412,0.976053,903.380765
min,1.000000,1.000000,0.010000
25%,1.000000,1.000000,24.485000
50%,1.000000,1.000000,48.890000
75%,1.000000,1.000000,108.470000
max,395.000000,36.000000,77113.430000


Looks like there are two a big outliers in there.  Before Moving on, we should remove those. 

In [54]:
data.loc[data['Revenue'] > data['Revenue'].quantile(.99)]

,fullvisitorId,operatingSystem,Paid,Visits,Transactions,Revenue
2167,7965265156545901045,Windows,False,6,1.0,4847.60
3174,79204932396995037,Windows,False,8,2.0,7047.15
20585,428994201200499894,Macintosh,True,26,3.0,3286.57
25297,6147396474895233852,Chrome OS,False,10,5.0,3265.28
26311,4604965471651937146,Chrome OS,False,3,1.0,4280.70
...,...,...,...,...,...,...
670435,0095747507496204221,Chrome OS,False,5,1.0,2365.50
670446,7113011772090059658,Macintosh,False,6,3.0,3454.41
672381,9817395183603701442,Macintosh,False,4,1.0,4085.50
688646,7435052038545127483,Macintosh,False,32,6.0,1871.57


In [55]:
data = data.loc[(data['Revenue'] < data['Revenue'].quantile(.99)) | data['Revenue'].isna()]

Let's check vor Outliers in `Visits` as well. 

In [56]:
data.loc[data['Visits'] > data['Visits'].quantile(.999)]

,fullvisitorId,operatingSystem,Paid,Visits,Transactions,Revenue
2,5000278396028518256,Windows,False,48,NaN,NaN
25,7150788234103617625,Macintosh,False,26,NaN,NaN
236,6278147906633575997,Chrome OS,False,21,NaN,NaN
246,315017261115039181,Macintosh,False,22,1.0,299.31
318,7956068088911744651,Windows,False,76,NaN,NaN
...,...,...,...,...,...,...
693150,6731418263426071704,Macintosh,False,27,NaN,NaN
695596,8843025310060521600,Linux,False,21,2.0,283.35
697325,4941770276769705740,Macintosh,False,23,NaN,NaN
698442,056927584469366764,Macintosh,False,22,NaN,NaN


In [57]:
data = data.loc[(data['Visits'] < data['Visits'].quantile(.999)) | (data['Visits'].isna())]

In [58]:
len(data)

707448

Now Let's aggregate the data to look for any patterns insights in Paid traffic.

In [59]:

# Group data by operating system and the paid boolean. 
agg_data = data.groupby(['operatingSystem', 'Paid']).agg({'fullvisitorId':'count', 'Visits':'sum', 'Transactions':'sum', 'Revenue':'sum'})
# Calculate the sucess rate of Unique visitors to transactions 
agg_data['rate'] = agg_data['Transactions']/agg_data['Visits']
# Create Average Order size from Revenue and Transactions
agg_data['Ave_Order'] = round((agg_data['Revenue']/agg_data['Transactions']), 2)
# Reformat Revenue to readable value
agg_data['Revenue'] = round(agg_data['Revenue'], 2)

In [60]:
agg_data.reset_index(inplace=True)

In [61]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=agg_data.loc[agg_data['Paid']==True]['operatingSystem'],
        y=agg_data.loc[agg_data['Paid']==True]['rate'],
        name="From Paid Source"
    )
)
fig.add_trace(
    go.Bar(
        x=agg_data.loc[agg_data['Paid']==False]['operatingSystem'],
        y=agg_data.loc[agg_data['Paid']==False]['rate'],
        name="From Organic Source"
    )
)
fig.update_layout(title='Conversion Rate by Operating System')
fig.update_yaxes(tickformat='%')
fig.show(renderer='notebook_connected')

In [62]:
# Send Figure to Plotly Chart Studio to share on embed
# py.plot(fig, filename = 'udacity-conversion_by_os', sharing='public')

Now lets loot at Average order size. 

In [63]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=agg_data.loc[agg_data['Paid']==True]['operatingSystem'],
        y=agg_data.loc[agg_data['Paid']==True]['Ave_Order'],
        name="From Paid Source"
    )
)
fig.add_trace(
    go.Bar(
        x=agg_data.loc[agg_data['Paid']==False]['operatingSystem'],
        y=agg_data.loc[agg_data['Paid']==False]['Ave_Order'],
        name="From Organic Source"
    )
)
fig.update_layout(title='Average Order by Operating System')
fig.update_yaxes(tickformat='$')
fig.show(renderer='notebook_connected')

In [64]:
# Send Figure to Plotly Chart Studio to share on embed
# py.plot(fig, filename = 'udacity_aveOrder_by_os', sharing='public')

Let's blow the whole thing up. We may have seen paid traffic increase conversion. We also saw paid traffic increase average ordersize. However, what about raw volume? 

In [65]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=agg_data.loc[agg_data['Paid']==True]['operatingSystem'],
        y=agg_data.loc[agg_data['Paid']==True]['Revenue'],
        name="From Paid Source"
    )
)
fig.add_trace(
    go.Bar(
        x=agg_data.loc[agg_data['Paid']==False]['operatingSystem'],
        y=agg_data.loc[agg_data['Paid']==False]['Revenue'],
        name="From Organic Source"
    )
)
fig.update_layout(title='Total Revenue by Operating System')
fig.update_yaxes(tickformat='$')
fig.show(renderer='notebook_connected')

In [66]:
# Send Figure to Plotly Chart Studio to share on embed
# py.plot(fig, filename = 'udacity-totRev_by_os', sharing='public')